In [1]:
from services.data_service import load_site_data, load_coef_prior_samples
from solvers.stan import _theta_reg_data, _prior_hyperparams
import numpy as np

Importing MCMC Module: mcmc_sampling_original.py


In [2]:
# Load sites' data
(
    zbar_2017,
    gamma_vals,
    z_2017,
    forestArea_2017_ha,
    theta_vals,
    gamma_coe,
    theta_coe,
    gamma_vcov_array,
    theta_vcov_array,
    theta_data,
    gamma_data,
) = load_site_data(
    10,
    norm_fac=1e11,
)

Data successfully loaded from '/Users/patriciohernandez/Projects/project-amazon/data/hmc/hmc_10SitesModel.csv'


In [3]:
theta_data

,id,X1,historical_precip,historical_temp,I.historical_temp.2.,lat,I.lat.2.,cattleSlaughter_farmGatePrice_2017,distance,zbar_2017_muni,log_cattleSlaughter_valuePerHa_2017
0,1,1.0,3.064932,0.808000,0.805915,0.845363,0.840704,-0.398258,1.421911,1.181881e+06,101.890458
1,2,1.0,3.064932,0.808000,0.805915,0.845363,0.840704,-0.398258,1.421911,1.181881e+06,101.890458
2,1,1.0,2.156587,0.205267,0.190646,0.341329,0.318339,0.493217,1.728454,7.068398e+06,171.473280
3,5,1.0,2.156587,0.205267,0.190646,0.341329,0.318339,0.493217,1.728454,7.068398e+06,171.473280
4,1,1.0,3.157149,0.962881,0.965922,0.862526,0.858738,-0.345103,1.234469,1.554863e+06,68.561741
...,...,...,...,...,...,...,...,...,...,...,...
590,10,1.0,-1.004162,-2.424435,-2.355767,-1.722755,-1.674927,0.686724,0.104464,2.018173e+05,1899.663230
591,10,1.0,-0.792480,-1.862329,-1.830330,-1.720168,-1.672576,2.416262,-0.808679,1.144672e+05,915.330578
592,10,1.0,-0.694920,-2.476114,-2.403560,-1.570791,-1.536175,1.582056,-0.356172,1.287097e+06,2658.845730
593,10,1.0,-1.164310,-1.058766,-1.061373,-1.496970,-1.468313,0.602710,0.550362,5.881073e+05,1819.620817


In [4]:
y_theta, X_theta, N_theta, K_theta, G_theta = _theta_reg_data(10, theta_data)

In [6]:
N_theta

595

In [7]:
I = _prior_hyperparams(y_theta, X_theta, "gamma")["inv_Lambda_gamma"]

In [8]:
mu = _prior_hyperparams(y_theta, X_theta, "gamma")["mu_gamma"]

In [ ]:
def gamma_fitted(gamma_coe, gamma_dataframe):
    # Copy df
    gamma_data = gamma_dataframe.copy()

    # Compute fitted values
    gamma_data["fitted_value"] = np.exp(
        (gamma_data.iloc[:, 1:6] * gamma_coe).sum(axis=1)
    )

    # Subset columns
    gamma_data = gamma_data[["id", "fitted_value"]]

    # Group by id and compute weighted mean for each group
    result = (
        gamma_data.groupby("id")["fitted_value"]
        .mean()
        .reset_index(name="gamma2017_Sites")
    )
    return result["gamma2017_Sites"].to_numpy()

In [ ]:
def new_gamma_fitted(gamma_coe, gamma_dataframe):
    # Copy df
    gamma_data = gamma_dataframe.copy()

    # Compute fitted values
    fitted = np.exp((gamma_data.iloc[:, 1:6] * gamma_coe).sum(axis=1))

    # Averaging matrix
    G = np.array([(gamma_data["id"].to_numpy() == i).astype(int) for i in range(1, 11)])
    G = G / G.sum(axis=1, keepdims=True)
    print(G)
    return G @ fitted

In [ ]:
gamma_fitted(gamma_coe, gamma_data)

In [ ]:
new_gamma_fitted(gamma_coe, gamma_data)

In [ ]:
def theta_fitted(theta_coe, theta_dataframe):
    # Copy df
    theta_data = theta_dataframe.copy()

    # Compute fitted values
    theta_data["fitted_value"] = np.exp(
        (theta_data.iloc[:, 1:9] * theta_coe).sum(axis=1)
    )

    # Subset and filter
    theta_data = theta_data[["id", "zbar_2017_muni", "fitted_value"]]
    theta_data = theta_data[theta_data["zbar_2017_muni"].notna()]

    # Weighted average function
    aux_price_2017 = 44.9736197781184

    def weighted_mean(group):
        return np.average(
            group["fitted_value"] / aux_price_2017, weights=group["zbar_2017_muni"]
        )

    # Take weighted average
    result = (
        theta_data.groupby("id")
        .apply(weighted_mean)
        .reset_index(name="theta2017_Sites")
    )

    # Return as numpy array
    return result["theta2017_Sites"].to_numpy()

In [ ]:
def new_theta_fitted(theta_coe, theta_dataframe):
    # Copy df
    theta_data = theta_dataframe.copy()

    # Filter out NA
    theta_data = theta_data[theta_data["zbar_2017_muni"].notna()]

    # Compute fitted values
    fitted = np.exp((theta_data.iloc[:, 1:9] * theta_coe).sum(axis=1))

    # Weighted average function
    aux_price_2017 = 44.9736197781184

    # Averaging matrix
    G = np.array([(theta_data["id"].to_numpy() == i).astype(int) for i in range(1, 11)])

    # Multily by weights
    G = theta_data["zbar_2017_muni"].to_numpy() * G
    G = G / G.sum(axis=1, keepdims=True)

    print(G.shape)
    return (G @ fitted) / aux_price_2017

In [ ]:
theta_fitted(theta_coe=theta_coe, theta_dataframe=site_theta_2017_df).flatten()

In [ ]:
new_theta_fitted(theta_coe=theta_coe, theta_dataframe=site_theta_2017_df).flatten()